In [0]:
pip install mlflow

Python interpreter will be restarted.
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139099 sha256=176820fc986ba22954eb6c4b6503c05cf44865a7fc02c51fa39343a19750e9b4
  Stored in directory: /root/.cache/pip/wheels/58/40/7c/d021d51dac18bfd095fb6837572ad2e6f1a34d221f4b1d976b
Successfully built databricks-cli
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-0dcc49a9-6c11-40cf-848c-c699689b09d0
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Not uninstalling urllib3 at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-0dcc49a9-6c11-40cf-848c-c699689b09d0
    Can't uninstall 'urllib3'. No files were found to 

In [0]:
%scala
spark.sparkContext.hadoopConfiguration.set(
  "fs.azure.account.key.bovianalytics.blob.core.windows.net",
  "a3/QhTcHik/gwnSCI/eBdjK1GruaFdeELpSzdnwKAKe6LebTUz6Ca4cKwUUWFdWO2JmUAFdaq/N6VMc/OXLdPg=="
)

In [0]:
from pyspark.sql.functions import to_date,col,avg
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType, TimestampType
import pyspark.sql.functions as F
from pyspark.sql import Window
from pprint import pprint as pp
import pandas as pd
import json
from urllib.request import  urlopen
import requests
import statsmodels.api as sm

#Read TrainTestData

In [0]:
TrainDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TrainDf125/')

In [0]:
TestDf = spark. \
  read. \
  parquet('wasbs://gpluse-cluster-2@bovianalytics.blob.core.windows.net/Projects/ChenYoungYan/11022022/Output/0405Split/TrainTestData/TestDf125/')

In [0]:
x_train = TrainDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_train = TrainDf.select("Decay305Vetor").\
    rdd.flatMap(lambda x: x).collect()

x_test = TestDf.select("features").\
    rdd.flatMap(lambda x: x).collect()
y_test = TestDf.select("Decay305Vetor").\
    rdd.flatMap(lambda x: x).collect()


featureList =['MultiparousCow','Winter','Spring','Autumn','Magnitude','TimeToPeakYield','Offset','Decay','TestDayMilkYield','AgeInMonths','HM305','MeanMagnitude', 'MeanTimeToPeakYield', 'MeanOffset', 'MeanDecay']

# check reference

In [0]:
sorted(set([(i[0], i[1]) for i in TrainDf.select("CalvingSeason", "CalvingSeasonEncode").collect()]),
    key=lambda x: x[0])

Out[8]: [('Autumn', SparseVector(3, {0: 1.0})),
 ('Spring', SparseVector(3, {2: 1.0})),
 ('Summer', SparseVector(3, {})),
 ('Winter', SparseVector(3, {1: 1.0}))]

In [0]:
sorted(set([(i[0], i[1]) for i in TestDf.select("CalvingSeason", "CalvingSeasonEncode").collect()]),
    key=lambda x: x[0])

Out[9]: [('Autumn', SparseVector(3, {0: 1.0})),
 ('Spring', SparseVector(3, {})),
 ('Summer', SparseVector(3, {2: 1.0})),
 ('Winter', SparseVector(3, {1: 1.0}))]

In [0]:
sorted(set([(i[0], i[1]) for i in TrainDf.select("ParityGroup","ParityGroupEncode").collect()]),
    key=lambda x: x[0])

Out[10]: [('MultiparousCow', SparseVector(1, {0: 1.0})),
 ('PrimiparousCow', SparseVector(1, {}))]

In [0]:
sorted(set([(i[0], i[1]) for i in TestDf.select("ParityGroup","ParityGroupEncode").collect()]),
    key=lambda x: x[0])

Out[11]: [('MultiparousCow', SparseVector(1, {0: 1.0})),
 ('PrimiparousCow', SparseVector(1, {}))]

# Model building

In [0]:
import mlflow
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import numpy as np
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error,mean_absolute_error

## Linear Regression

In [0]:
X = sm.add_constant(x_train)
mod = sm.OLS(y_train,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     411.6
Date:                Sun, 23 Oct 2022   Prob (F-statistic):               0.00
Time:                        13:44:50   Log-Likelihood:                 61990.
No. Observations:               10277   AIC:                        -1.239e+05
Df Residuals:                   10261   BIC:                        -1.238e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0019   3.88e-05     48.797      0.0

### train

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    
    # model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
    coeffs = lm.coef_.tolist()[0]
    y_pred = lm.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    
    plt.figure(figsize=(20,7))
    plt.bar(featureList,coeffs)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.ylabel('β', fontsize=20,rotation=0)
    plt.xticks(rotation=25,fontsize= 13)
    plt.yticks(fontsize= 13)
    plt.savefig("β.png",bbox_inches='tight')
    plt.close()
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))
    mlflow.log_artifact("β.png")

### test

In [0]:
with mlflow.start_run(run_name = 'Linear Regression') as run:
    
    # train the model
    
    lm = LinearRegression()
    lm.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = lm.predict(x_test)
   
    # log parameters
    mlflow.log_param('data','test')
    mlflow.log_param('type','whole')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

## Ridge Regression

### train

#### CV

In [0]:

with mlflow.start_run(run_name = 'RidgeRegression') as run:
    
    
    # model
    
    RR = Ridge()
    
     # log parameters
    mlflow.log_param('type','CV')
    
    # cross validation
    #tag
    
    alpha_values = {'alpha':[i*0.1  for i in range(90,120,2)]}
    mlflow.set_tags(alpha_values)
    
    grid_search = GridSearchCV(RR, 
                           alpha_values,
                           scoring=["r2",'neg_root_mean_squared_error'],
                           cv=10,
                           refit='r2', 
                           return_train_score=True,
                           n_jobs=-1)
    
    grid_search.fit(x_train,y_train)
    

    # log parameters
    mlflow.log_param('best parameters',grid_search.best_params_)
    mlflow.log_param('best estimator',grid_search.best_estimator_)
    mlflow.log_param('best score',grid_search.best_score_)
    
    
    plt.plot(grid_search.cv_results_['mean_test_neg_root_mean_squared_error'].tolist())
    plt.savefig("test_rmse.png")
    plt.close()
    
    plt.plot(grid_search.cv_results_['mean_test_r2'].tolist())
    plt.savefig("test_r2.png")
    plt.close()
    
     # log artifact
    
    mlflow.log_artifact("test_r2.png")
    mlflow.log_artifact("test_rmse.png")    
    

#### train_whole

In [0]:
with mlflow.start_run(run_name = 'RidgeRegression') as run:
      
    # model
    RR = Ridge(**grid_search.best_params_)
    RR.fit(x_train,y_train)
    y_pred = RR.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))

### test

In [0]:
with mlflow.start_run(run_name = 'Ridge Regression') as run:
    
    RR = Ridge(**grid_search.best_params_)
    
    # train the model
    
    RR = Ridge(**grid_search.best_params_)
    RR.fit(x_train,y_train)
       
    # use the model to predict the test data
    
    y_test_pred = RR.predict(x_test)
   
    # log parameters
    mlflow.log_param('type','test')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

## Lasso Regression

### train

#### CV

In [0]:

with mlflow.start_run(run_name = 'LassoRegression') as run:
    
    # model
    
    LS = Lasso()
    
     # log parameters
    mlflow.log_param('type','CV')
    
    # cross validation
    
    alpha_values = {'alpha':[i*0.00001 for i in range(0,100,10)]}
    mlflow.set_tags(alpha_values)
    
    grid_search_ls = GridSearchCV(LS, 
                           alpha_values,
                           scoring=["r2",'neg_root_mean_squared_error'],
                           cv=10,
                           refit='r2', 
                           return_train_score=True,
                           n_jobs=-1)
    
    grid_search_ls.fit(x_train,y_train)

    # log parameters
    mlflow.log_param('best parameters',grid_search_ls.best_params_)
    mlflow.log_param('best estimator',grid_search_ls.best_estimator_)
    mlflow.log_param('best score',grid_search_ls.best_score_)
    
    
    plt.plot(grid_search_ls.cv_results_['mean_test_neg_root_mean_squared_error'].tolist())
    plt.savefig("test_rmse.png")
    plt.close()
    
    plt.plot(grid_search_ls.cv_results_['mean_test_r2'].tolist())
    plt.savefig("test_r2.png")
    plt.close()
    
     # log artifact
    
    mlflow.log_artifact("test_r2.png")
    mlflow.log_artifact("test_rmse.png")
    

/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  original_result = original(self, *args, **kwargs)
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017342599329817207, tolerance: 5.555311432212362e-07
  model = cd_fast.enet_coordinate_descent(


#### train_whole

In [0]:
with mlflow.start_run(run_name = 'LassoRegression') as run:
    
    mlflow.set_tags(grid_search_ls.best_params_)
    # model
    
    LS = Lasso(**grid_search_ls.best_params_)
    LS.fit(x_train,y_train)
    y_pred = LS.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))

/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  original_result = original(self, *args, **kwargs)
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017342599329817207, tolerance: 5.555311432212362e-07
  model = cd_fast.enet_coordinate_descent(


### test

In [0]:
with mlflow.start_run(run_name = 'LassoRegression') as run:
    
    # train the model
    mlflow.set_tags(grid_search_ls.best_params_)
    
    LS = Lasso(**grid_search_ls.best_params_)
    LS.fit(x_train,y_train)
    
    y_test_pred = LS.predict(x_test)
   
    # log parameters
    mlflow.log_param('type','test')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    

/databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_sklearn.py:29: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  original_result = original(self, *args, **kwargs)
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017342599329817207, tolerance: 5.555311432212362e-07
  model = cd_fast.enet_coordinate_descent(


## Random Forest

### train

#### CV

In [0]:
#grid
with mlflow.start_run(run_name = 'RandomForest') as run:
    
    # model
    
    RF = RandomForestRegressor()
    
     # log parameters
    mlflow.log_param('type','CV')
        
   # params = [{'n_estimators':[i for i in range(10,200,30)],#Number of Trees
              # 'max_depth':[i for i in range(1,30,1)],# Tree Depth
             #   'max_features':[i for i in range(1,18,1)],
            #  'min_samples_split': [i for i in range(2,20,1)], #The minimum number of samples required to split an internal node
            #   'min_samples_leaf': [i for i in range(1,20,1)], #The minimum number of samples required to be at a leaf node
           
    
    params = [{'n_estimators':[126],#Number of Trees
              'max_depth':[13],# Tree Depth
             'max_features':[3],
             'min_samples_split': [5], #The minimum number of samples required to split an internal node
             'min_samples_leaf': [i for i in range(1,20,1)], #The minimum number of samples required to be at a leaf node
           
             
              }
             ]
    
    mlflow.set_tags(params[0])
    
    grid_search = GridSearchCV (RF, 
                           param_grid= params,
                           scoring=["r2",'neg_root_mean_squared_error'],
                           cv=10,        
                           refit='r2', 
                           return_train_score=True,
                           n_jobs=-1)
    
    grid_search.fit(x_train,np.ravel(y_train))
    
    # log parameters
    mlflow.log_param('best parameters',grid_search.best_params_)
    mlflow.log_param('best estimator',grid_search.best_estimator_)
    mlflow.log_param('best score',grid_search.best_score_)
    
    
    plt.plot(grid_search.cv_results_['mean_test_neg_root_mean_squared_error'].tolist())
    plt.savefig("test_rmse.png")
    plt.close()
    
    plt.plot(grid_search.cv_results_['mean_test_r2'].tolist())
    plt.savefig("test_r2.png")
    plt.close()
    
     # log artifact
    
    mlflow.log_artifact("test_r2.png")
    mlflow.log_artifact("test_rmse.png")


#### train_whole

In [0]:
with mlflow.start_run(run_name = 'RandomForest') as run:
    
    mlflow.set_tags(grid_search.best_params_)
    # model
    
    RF = RandomForestRegressor(**grid_search.best_params_)
    RF.fit(x_train,np.ravel(y_train))
    y_pred = RF.predict(x_train)
   
    # log parameters
    mlflow.log_param('data','train')
    mlflow.log_param('type','whole')
    
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_train,y_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_train,y_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_train,y_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_train,y_pred))   
    

### test

In [0]:
with mlflow.start_run(run_name = 'RandomForest') as run:
    
    RF = RandomForestRegressor(**grid_search.best_params_)
    RF.fit(x_train,np.ravel(y_train))
       
    # use the model to predict the test data
    
    y_test_pred = RF.predict(x_test)
   
    # log parameters
    mlflow.log_param('type','test')
    
     # log metrics
    mlflow.log_metric('rmse',mean_squared_error(y_test,y_test_pred, squared=False))
    mlflow.log_metric('r2',r2_score(y_test,y_test_pred))
    mlflow.log_metric('mape',mean_absolute_percentage_error(y_test,y_test_pred))
    mlflow.log_metric('mae',mean_absolute_error(y_test,y_test_pred))
    